In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
ROOT_DIR = '/content/gdrive/My Drive/Yolov8Developer/Yolov8Dataset'

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
!yolo export model=/content/gdrive/MyDrive/Yolov8Developer/Yolov8Dataset/yolov8n.pt format=torchscript

In [ ]:
import os
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.yaml')  # build a new model from scratch
model = YOLO('/content/gdrive/MyDrive/Yolov8Developer/Yolov8Dataset/yolov8n.pt')  # load a pretrained model (recommended for training)

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "coco128-1.yaml"), epochs=5)  # train the model
results = model.val()  # evaluate model performance on the validation set
results = model.export(format='onnx')  # export the model to ONNX format

In [ ]:
!scp -r /content/runs '/content/gdrive/My Drive/ComputerVisionDeveloper/TrainYolov8CustomDataset'

# Colab to Drive (exchange)

In [ ]:
!mv datasets "/content/gdrive/My Drive/ComputerVisionDeveloper/TrainYolov8CustomDataset"

In [ ]:
!cp -r "/content/gdrive/MyDrive/ComputerVisionDeveloper/TrainYolov8CustomDataset/datasets" "/content/"

# Route

In [ ]:
import os

for root, dirs, files in os.walk('/'):
    if 'yolov8n.pt' in files:
        print(os.path.join(root, 'yolov8n.pt'))

# Pre-trained

In [ ]:
# Download COCO val
import torch
torch.hub.download_url_to_file('https://ultralytics.com/assets/coco2017val.zip', 'tmp.zip')  # download (780M - 5000 images)
!unzip -q tmp.zip -d datasets && rm tmp.zip  # unzip

In [ ]:
# Train YOLOv8n on COCO8 for 10 epochs
!yolo train model=yolov8n.pt data=coco8.yaml epochs=10 imgsz=640